## Seq to Seq Modelling : Hindi English Translation

In [70]:
import numpy as np
np.random.seed(42)

In [ ]:
local_path="G:/UpX Academy/Deep Learning/Week6SeqToSeq/"

In [ ]:
import zipfile
import io

In [ ]:
zf = zipfile.ZipFile(local_path+'hin-eng.zip', 'r')

In [ ]:
data = ''
with zf.open('hin.txt') as readfile:
  for line in io.TextIOWrapper(readfile, 'utf-8'):
    data += line

In [ ]:
data

In [ ]:
data =  data.split('\n')

In [ ]:
data

In [ ]:
data[0]

### Review data

In [ ]:
len(data)

### Separate out enoder and decoder input

In [ ]:
encoder_text = []

In [ ]:
decoder_text = []

In [ ]:
for line in data:
    try:
        in_txt, out_txt = line.split('\t')
        encoder_text.append(in_txt)
        
        # Add tab '<start>' as 'start sequence in target
        # And '<end>' as End
        decoder_text.append('<start> ' + out_txt + ' <end>')
    except:
        pass #ignore data which goes into error  

In [ ]:
decoder_text[105:110]

In [ ]:
encoder_text[105:110]

### Build input and output sequences

In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

encoder Tokenizer

In [ ]:
encoder_t = Tokenizer()

In [ ]:
encoder_t.fit_on_texts(encoder_text)

In [ ]:
encoder_seq = encoder_t.texts_to_sequences(encoder_text)

In [ ]:
encoder_seq[105:110]

In [ ]:
max_encoder_seq_length = max([len(txt) for txt in encoder_seq])
max_encoder_seq_length

In [ ]:
encoder_vocab_size = len(encoder_t.word_index)

In [ ]:
encoder_vocab_size

Decoder Tokenizer

In [ ]:
decoder_t = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')

In [ ]:
decoder_t.fit_on_texts(decoder_text)

In [ ]:
decoder_seq = decoder_t.texts_to_sequences(decoder_text)

In [ ]:
decoder_seq[105:110]

In [ ]:
max_decoder_seq_length = max([len(txt) for txt in decoder_seq])
max_decoder_seq_length

In [ ]:
decoder_vocab_size = len(decoder_t.word_index)

In [ ]:
decoder_vocab_size

### Pad Sequences

In [ ]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [ ]:
encoder_input_data = pad_sequences(encoder_seq, maxlen=max_encoder_seq_length, padding='post')

In [ ]:
decoder_input_data = pad_sequences(decoder_seq, maxlen=max_decoder_seq_length, padding='post')

In [99]:
encoder_input_data.shape

(2808, 22)

In [100]:
encoder_input_data[1456]

array([  38,  743,    8,  519, 1085,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

In [ ]:
decoder_input_data.shape

Integer to Word Converter for Decoder Data

In [ ]:
int_to_word_decoder = dict((i,c) for c, i in decoder_t.word_index.items())

In [ ]:
int_to_word_decoder[15]

## Build Decoder Output

In [ ]:
decoder_target_data = np.zeros((decoder_input_data.shape[0], decoder_input_data.shape[1]))

In [ ]:
decoder_target_data.shape

In [ ]:
for i in range(decoder_input_data.shape[0]):
    for j in range(1,decoder_input_data.shape[1]):
        decoder_target_data[i][j-1] = decoder_input_data[i][j]  

In [ ]:
decoder_input_data[0]

In [ ]:
decoder_target_data[0]

Convert target data into one hot encode

In [ ]:
from tensorflow.python.keras.utils import  to_categorical

In [ ]:
decoder_target_one_hot = np.zeros((decoder_input_data.shape[0], 
                                   decoder_input_data.shape[1],
                                   len(decoder_t.word_index)+1))

In [ ]:
for i in range(decoder_target_data.shape[0]):
    for j in range(decoder_target_data.shape[1]):
        decoder_target_one_hot[i][j] = to_categorical(decoder_target_data[i][j],
                                                      num_classes=len(decoder_t.word_index)+1) 

In [ ]:
decoder_target_one_hot.shape

In [ ]:
decoder_target_one_hot[0:1]

## Building the Training Model

In [ ]:
from tensorflow.python.keras.layers import Input, LSTM, Dense, Embedding

In [ ]:
#define config parameters
encoder_embedding_size = 50
decoder_embedding_size = 50
rnn_units = 256

Build Encoder

In [ ]:
encoder_inputs = Input(shape=(None,))

In [ ]:
encoder_inputs.shape

In [ ]:
encoder_embedding = Embedding(encoder_vocab_size+1, encoder_embedding_size)

In [ ]:
encoder_embedding_output = encoder_embedding(encoder_inputs)

In [ ]:
x, state_h, state_c = LSTM(rnn_units,return_state=True)(encoder_embedding_output)

In [ ]:
state_c.graph

In [ ]:
encoder_states = [state_h, state_c]

Build Decoder

In [ ]:
decoder_inputs = Input(shape=(None,))

In [ ]:
decoder_embedding = Embedding(decoder_vocab_size + 1, decoder_embedding_size)

In [ ]:
decoder_embedding_output = decoder_embedding(decoder_inputs)

In [ ]:
decoder_rnn = LSTM(rnn_units, return_sequences=True, return_state=True)

In [ ]:
#Initialize initial state with encoder_states
#Output will be all hidden sequences, last 'h' state and last 'c' state
x,_,_ = decoder_rnn(decoder_embedding_output, initial_state=encoder_states)

In [ ]:
decoder_dense = Dense(decoder_vocab_size + 1, activation='softmax')

In [ ]:
decoder_outputs = decoder_dense(x)

## Build model using both Encoder and Decoder

In [ ]:
from tensorflow.python.keras.models import Model

In [ ]:
model = Model([encoder_inputs, decoder_inputs], #2 Inputs to the model
              decoder_outputs) #Output of the model

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [71]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     119450      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     149650      input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM

In [86]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_one_hot,
          batch_size=64,
          epochs=100,
          validation_split=0.2)

Train on 2246 samples, validate on 562 samples
Epoch 1/100
2246/2246 [==============================] - 59s 26ms/step - loss: 0.4002 - val_loss: 3.0672
Epoch 2/100
2246/2246 [==============================] - 59s 26ms/step - loss: 0.3963 - val_loss: 3.0636
Epoch 3/100
2246/2246 [==============================] - 59s 26ms/step - loss: 0.3891 - val_loss: 3.0684
Epoch 4/100
2246/2246 [==============================] - 59s 26ms/step - loss: 0.3856 - val_loss: 3.0648
Epoch 5/100
2246/2246 [==============================] - 60s 27ms/step - loss: 0.3788 - val_loss: 3.0862
Epoch 6/100
2246/2246 [==============================] - 59s 26ms/step - loss: 0.3707 - val_loss: 3.0922
Epoch 7/100
2246/2246 [==============================] - 59s 26ms/step - loss: 0.3680 - val_loss: 3.0896
Epoch 8/100
2246/2246 [==============================] - 59s 26ms/step - loss: 0.3615 - val_loss: 3.1322
Epoch 9/100
2246/2246 [==============================] - 59s 26ms/step - loss: 0.3557 - val_loss: 3.1170
Epoch 10

Epoch 78/100
2246/2246 [==============================] - 47s 21ms/step - loss: 0.1260 - val_loss: 3.5029
Epoch 79/100
2246/2246 [==============================] - 47s 21ms/step - loss: 0.1242 - val_loss: 3.4942
Epoch 80/100
2246/2246 [==============================] - 47s 21ms/step - loss: 0.1235 - val_loss: 3.5105
Epoch 81/100
2246/2246 [==============================] - 47s 21ms/step - loss: 0.1217 - val_loss: 3.5206
Epoch 82/100
2246/2246 [==============================] - 47s 21ms/step - loss: 0.1189 - val_loss: 3.5154
Epoch 83/100
2246/2246 [==============================] - 46s 21ms/step - loss: 0.1164 - val_loss: 3.5414
Epoch 84/100
2246/2246 [==============================] - 46s 21ms/step - loss: 0.1146 - val_loss: 3.5302
Epoch 85/100
2246/2246 [==============================] - 46s 21ms/step - loss: 0.1133 - val_loss: 3.5355
Epoch 86/100
2246/2246 [==============================] - 46s 21ms/step - loss: 0.1099 - val_loss: 3.5479
Epoch 87/100
2246/2246 [======================

In [ ]:
model.save('seq2seq_training_translation.hd5')

In [87]:
decoder_outputs

<tf.Tensor 'dense_1/truediv:0' shape=(?, ?, 2993) dtype=float32>

## Building Model for Prediction

### Build the encoder model to predict Encoder States

In [88]:
encoder_model = Model(encoder_inputs, encoder_states)

### Build the decoder model

In [89]:
decoder_state_input_h = Input(shape=(rnn_units,))

In [90]:
decoder_state_input_c = Input(shape=(rnn_units,))

In [91]:
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

Get RNN outputs,states(s)

In [92]:
x = decoder_embedding(decoder_inputs)

In [93]:
#We will use the layer which we trained earlier
rnn_outputs, state_h, state_c = decoder_rnn(x, initial_state=decoder_states_inputs)

In [94]:
#Why do we need this?
#We need this to predict the sequence of target language words
decoder_states = [state_h, state_c]

get decoder output

In [95]:
decoder_outputs = decoder_dense(rnn_outputs)

Build decoder model

In [96]:
decoder_model = Model([decoder_inputs] + decoder_states_inputs,  #Model inputs
                     [decoder_outputs] + decoder_states)

## Predicting Output

Build prediction function

In [97]:
def decode_sentence(input_sequence):
    
    #Get the encoder state values
    decoder_initial_states_value = encoder_model.predict(input_sequence)
    
    #Build a sequence with '<start>' - starting sequence for Decoder
    target_seq = np.zeros((1,1))    
    target_seq[0][0] = decoder_t.word_index['<start>']
    
    #flag to check if prediction should be stopped
    stop_loop = False
    
    #Initialize predicted sentence
    predicted_sentence = ''
    
    #start the loop
    while not stop_loop:
        
        predicted_outputs, h, c = decoder_model.predict([target_seq] + 
                                                        decoder_initial_states_value)
        
        #Get the predicted output with highest probability
        predicted_output = np.argmax(predicted_outputs[0,-1,:])
        
        #Get the predicted word from predicter integer
        predicted_word = int_to_word_decoder[predicted_output]
        
        #Check if prediction should stop
        if(predicted_word == '<end>' or len(predicted_sentence) > max_decoder_seq_length):
            
            stop_loop = True
            continue
                    
        #Updated predicted sentence
        if (len(predicted_sentence) == 0):
            predicted_sentence = predicted_word
        else:
            predicted_sentence = predicted_sentence + ' ' + predicted_word
            
        #Update target_seq to be the predicted word index
        target_seq[0][0] = predicted_output
        
        #Update initial states value for decoder
        decoder_initial_states_value = [h,c]
        
    
    return predicted_sentence

Call Prediction function

In [98]:
#Get a random sentence
start_num = np.random.randint(0, high=len(encoder_text) - 10)
print(start_num)

for i in range(start_num, start_num + 10):
    input_seq = encoder_input_data[i : i+1]
    predicted_sentence = decode_sentence(input_seq)
    print('--------')
    print ('Input sentence: ', encoder_text[i])
    print ('Predicted sentence: ', predicted_sentence )

1373
--------
Input sentence:  My telephone is out of order.
Predicted sentence:  मेरा फ़ोन खराब है।
--------
Input sentence:  Now they have three children.
Predicted sentence:  अब उनके तीन बच्चे हैं।
--------
Input sentence:  One language is never enough.
Predicted sentence:  एक भाषा कभी काफ़ी नहीं होती।
--------
Input sentence:  Please make yourself at home.
Predicted sentence:  इसको अपना घर ही समझो।
--------
Input sentence:  Please make yourself at home.
Predicted sentence:  इसको अपना घर ही समझो।
--------
Input sentence:  Please show me your notebook.
Predicted sentence:  मुझे लगता है कि तुम ग़लत हो।
--------
Input sentence:  Please wait for five minutes.
Predicted sentence:  कृपया पाँच मिनट इंतेज़ार करें।
--------
Input sentence:  Say which one you would like.
Predicted sentence:  बोलो तुम्हे कौनसा चाहिए।
--------
Input sentence:  Seicho Matumoto died in 1992.
Predicted sentence:  सेइचो मात्सुमोतो का निधन सन १९९२
--------
Input sentence:  She died yesterday afternoon.
Predicted sen

## Save prediction model and tokenizers

In [102]:
#Save encoder and decoder model for Prediction
encoder_model.compile(optimizer='adam', loss='mse')
decoder_model.compile(optimizer='adam', loss='categorical_crossentropy')
encoder_model.save('G:/UpX Academy/Deep Learning/Week6SeqToSeq/seq2seq_encoder_eng_hin.hd5')
decoder_model.save('G:/UpX Academy/Deep Learning/Week6SeqToSeq/seq2seq_decoder_eng_hin.hd5')

In [103]:
#Save tokenizers
import pickle

pickle.dump(encoder_t,open('G:/UpX Academy/Deep Learning/Week6SeqToSeq/encoder_tokenizer_eng','wb'))
pickle.dump(decoder_t,open('G:/UpX Academy/Deep Learning/Week6SeqToSeq/decoder_tokenizer_hin','wb'))